# LSTM Architecture for predicting the volatililty

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import tensorflow as tf

from sklearn.compose import make_column_transformer
from sklearn.metrics import r2_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

from tensorflow import keras


In [2]:
list_order_book_file_train = glob.glob('optiver-realized-volatility-prediction/book_train.parquet/*')

In [3]:
train = pd.read_csv('optiver-realized-volatility-prediction/train.csv')

# Calculate additional features

Calculate the features from the Optiver examples provided to us by Optiver

In [4]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def add_features_and_aggregate_data(df):
    # spread between ask and bis price on first level in order book
    df['price_spread_l1'] = df['ask_price1'] - df['bid_price1']
    # added price spread as log difference to make it independent
    df['price_spread_l1_log_diff'] = df.groupby('time_id')['price_spread_l1'].transform(log_return)
    # I tried to aggregate the data in buckets of 50 seconds and called the bucket index timeslice
    # I would like to reduce memory consumption and train time with this approach
    # There is a maximum of 600(0-599) Seconds in every training bucket(stock_id, time_id)
    # So there should be a maximum of 12 Buckets
    df['timeslice'] = df['seconds_in_bucket'] // 50 
    # calculated the weighted average price
    df['wap'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1']+ df['ask_size1'])
    # calculated log return
    df['log_return'] = df.groupby(['time_id'], as_index=False)['wap'].transform(log_return)
    # drop rows with na. The na gets created by the diff function by calculating the log return
    # use inplace to save memory
    df.dropna(subset=['log_return', 'price_spread_l1_log_diff'], inplace=True)
    #calculate realized voltality for every bucket
    df['realized_vol'] = df.groupby(['time_id', 'timeslice'], as_index=False)['log_return'].transform(realized_volatility)
    
    return df.groupby(['time_id', 'timeslice']).agg(
                stock_id=('stock_id', 'max'),
                min_price_spread_l1_log_diff=('price_spread_l1_log_diff', 'min'),
                max_price_spread_l1_log_diff=('price_spread_l1_log_diff', 'max'),
                mean_price_spread_l1_log_diff=('price_spread_l1_log_diff', 'mean'),
                min_log_return=('log_return', 'min'),
                max_log_return=('log_return', 'max'),
                mean_log_return=('log_return', 'mean'),
                min_realized_vol=('realized_vol', 'min'),
                max_realized_vol=('realized_vol', 'max'),
                mean_realized_vol=('realized_vol', 'mean'),
    ).reset_index()

# LSTM

## Preprocess Input Data

In [5]:
feature_columns = ['stock_id', 'min_price_spread_l1_log_diff', 'max_price_spread_l1_log_diff', 
           'mean_price_spread_l1_log_diff', 'min_log_return', 'max_log_return',
           'mean_log_return', 'min_realized_vol', 'max_realized_vol', 'mean_realized_vol']

In [6]:
def get_input_data(list_file):
    df_input = pd.DataFrame()
    for file in list_file:
        # read only needed columns to save memory
        df_input_file = pd.read_parquet(file, 
                                        columns=['time_id', 'seconds_in_bucket', 
                                                 'bid_size1' ,'bid_price1', 
                                                 'ask_size1', 'ask_price1'])
        # get stock id from filename
        df_input_file['stock_id'] = int(file.split('=')[1])
        # add features and aggregate data
        df_input = pd.concat([df_input,
                              add_features_and_aggregate_data(df_input_file)], 
                              ignore_index=True, copy=False)
    return df_input

In [7]:
df_input = get_input_data(list_file=list_order_book_file_train)

In [8]:
# add row id and targets to data
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
df_input = df_input.merge(train, on=['time_id', 'stock_id'], how = 'left')

In [9]:
# split train and validation set groupwise by row-id
train_inds, val_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7)
                            .split(df_input, groups=df_input['row_id'])
                           )

train = df_input.iloc[train_inds]
validation = df_input.iloc[val_inds]

In [10]:
# fit transformer on columns in train dataset
column_transformer = make_column_transformer(
    (StandardScaler(), ['min_price_spread_l1_log_diff', 'max_price_spread_l1_log_diff', 
                        'mean_price_spread_l1_log_diff', 'min_log_return', 
                        'max_log_return', 'mean_log_return', 'min_realized_vol', 
                        'max_realized_vol', 'mean_realized_vol']),
    remainder='passthrough')
column_transformer = column_transformer.fit(train[feature_columns])


In [11]:
# save memory
del(df_input)

In [12]:
# reshape and transform columns groupwise to get the needed shape for LSTM [batch, timesteps, feature]
# Pad to length 12, the max of seconds_in_bucket/50, to get equal sized sequences.
train_np = np.array([keras.preprocessing.sequence.pad_sequences(
    column_transformer.transform(
        x[feature_columns]
    ).transpose(), 
    maxlen=12, 
    dtype='float32', 
    value=0.0).transpose() for _, x in train.groupby('row_id')])
val_np = np.array([keras.preprocessing.sequence.pad_sequences(column_transformer.transform(x[feature_columns]).transpose(), 
                                                               maxlen=12, 
                                                               dtype='float32',
                                                               value=0.0).transpose() for _, x in validation.groupby('row_id')])
# scale targets
target_scaler = StandardScaler()
target_train = target_scaler.fit_transform(
    train.groupby(['stock_id', 'time_id'])['target'].first().values.reshape(-1,1)
).reshape(-1)
target_val = target_scaler.transform(
    validation.groupby(['stock_id', 'time_id'])['target'].first().values.reshape(-1,1)
).reshape(-1)

In [13]:
# save memory
del(train, validation)

In [14]:
# some simple LSTM
# The architecture is mostly random. I don't know how to create a good architecture for this problem
learning_rate = 0.001

inputs_lstm = keras.layers.Input(shape=(train_np.shape[1], train_np.shape[2]))
masking = keras.layers.Masking(mask_value=0.0, input_shape=(train_np.shape[1], train_np.shape[2]))(inputs_lstm)
lstm_1_out = keras.layers.LSTM(128, return_sequences=True)(masking)
lstm_2_out = keras.layers.LSTM(64, return_sequences=True)(lstm_1_out)
lstm_3_out = keras.layers.LSTM(10, activation='relu')(lstm_2_out)
outputs = keras.layers.Dense(1)(lstm_3_out)

model = keras.Model(inputs=inputs_lstm, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_absolute_error')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 10)]          0         
_________________________________________________________________
masking (Masking)            (None, 12, 10)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 12, 128)           71168     
_________________________________________________________________
lstm_1 (LSTM)                (None, 12, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                3000      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 123,587
Trainable params: 123,587
Non-trainable params: 0
_______________________________________________________

In [15]:
# early stopping and fit function
def run_trainings_batch(dataset_train, target, val, epochs):
    path_checkpoint = 'model_checkpoint.h5'
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

    history = model.fit(
        dataset_train,
        target,
        epochs=epochs,
        batch_size=1000,
        validation_data=val,
        callbacks=[es_callback],
    )

In [16]:
# train model
run_trainings_batch(train_np, target_train, (val_np, target_val), 100)

Epoch 1/100
344/344 [==============================] - 94s 251ms/step - loss: 0.6345 - val_loss: 0.6309
Epoch 2/100
344/344 [==============================] - 84s 243ms/step - loss: 0.6224 - val_loss: 0.6329
Epoch 3/100
344/344 [==============================] - 84s 245ms/step - loss: 0.6201 - val_loss: 0.6234
Epoch 4/100
344/344 [==============================] - 84s 244ms/step - loss: 0.6147 - val_loss: 0.6225
Epoch 5/100
344/344 [==============================] - 83s 242ms/step - loss: 0.6115 - val_loss: 0.6182
Epoch 6/100
344/344 [==============================] - 84s 244ms/step - loss: 0.6110 - val_loss: 0.6202
Epoch 7/100
344/344 [==============================] - 85s 247ms/step - loss: 0.6109 - val_loss: 0.6172
Epoch 8/100
344/344 [==============================] - 86s 250ms/step - loss: 0.6063 - val_loss: 0.6122
Epoch 9/100
344/344 [==============================] - 85s 246ms/step - loss: 0.6055 - val_loss: 0.6085
Epoch 10/100
344/344 [==============================] - 85s 246m

In [17]:
# save memory
del(train_np, target_train, val_np, target_val)

# Making predictions

In [18]:
list_order_book_file_test = glob.glob('/optiver-realized-volatility-prediction/book_test.parquet/*')
# get prediction for every file
def get_predictions(list_file):
    prediction = pd.DataFrame()
    for file in list_file:
        df_input_test = get_input_data(list_file=[file])
        df_input_test['row_id'] = df_input_test['stock_id'].astype(str) + '-' + df_input_test['time_id'].astype(str)
        df_pred_np = np.array([keras.preprocessing.sequence.pad_sequences(
            column_transformer.transform(x[feature_columns]).transpose(), 
            maxlen=12, 
            dtype='float32', 
            value=0.0).transpose() for _, x in df_input_test.groupby('row_id')])
        prediction_new = pd.DataFrame()
        prediction_new['row_id'] = df_input_test['row_id'].unique()
        prediction_new['target'] = model.predict(df_pred_np).reshape(-1)
        prediction = pd.concat([prediction, prediction_new])
    prediction['target'] = target_scaler.inverse_transform(prediction['target'])
    return prediction


In [19]:
# save submission
get_predictions(list_file=list_order_book_file_test).to_csv('submission.csv',index = False)